IMPORT REQUIRED LIBRARIES

In [ ]:
import shap
from sklearn.ensemble import RandomForestClassifier
from explainerdashboard import ClassifierExplainer, ExplainerDashboard
from pycaret.classification import *
import optuna
import Reqs
import numpy as np
from sklearn.metrics import accuracy_score
import databricks.sql
from sqlalchemy import create_engine
import pandas as pd
import ipywidgets as widgets
from IPython.display import display
import ML_Model_Preprocessing_Loading as MMPL
import Model_Creation_Tools as MCL
import Model_Explanation_Tools as MET
import matplotlib.pyplot as plt
pd.set_option('display.max_columns',1000)
pd.set_option('display.max_rows',1000)

Fetch Training Data:

In [ ]:
data = MMPL.DRP_Agena_Ingestion_Model_Fetch_Data(segment='train')

Apply Preprocessing Pipeline

In [ ]:
testing = data

In [ ]:
test = testing.groupby('Model_y')['AnyPaymentReceived'].agg(['mean', 'count'])

In [ ]:
pd.set_option('display.max_rows',1000)

In [ ]:
test[test['count']>2].sort_values('mean',ascending=False).head(500)

In [ ]:
data, features, target  = MMPL.DRP_Agena_Ingestion_Model_Preprocess(data)

Setup ML Model Environment

In [ ]:
    car_ranking_dim = pd.read_csv('Car_ranking_dim.csv')
    data = pd.merge(data, car_ranking_dim, left_on='Make_y', right_on='Car Brand', how='inner')

In [ ]:
data

In [ ]:
MLENV, model = MCL.Create_Model(data,'AnyPaymentReceived','lightgbm',['Debt_id','Registration','BatchID'])

In [ ]:
MET.Model_Generate_Shap_Diagrams(model)

In [ ]:
model = tune_model(model, optimize='Precision')

In [ ]:
model = calibrate_model(model)

In [ ]:
model = finalize_model(model)

In [ ]:
dataTest = MMPL.DRP_Agena_Ingestion_Model_Fetch_Data(segment='live')

In [ ]:
dataTest, featuresTest, targetTest  = MMPL.DRP_Agena_Ingestion_Model_Preprocess(dataTest)

MODEL INTERPRETATION DASHBOARD:

dashboard(
    best,
    run_kwargs={"host": "127.0.0.1"},
    display_format= 'external'
     )

MODEL CALIBRATION:

In [ ]:
car_ranking_dim = pd.read_csv('Car_ranking_dim.csv')
dataTest = pd.merge(dataTest, car_ranking_dim, left_on='Make_y', right_on='Car Brand', how='inner')

In [ ]:
probability_threshold = 0.85
percentile_cutoff = 0.85    
predictions = predict_model(model, data=dataTest,probability_threshold=probability_threshold)
predictions = predictions[predictions['prediction_label']==1

predictions = predictions.sort_values('prediction_score', ascending=False)
cutoff_index = int(len(predictions) * percentile_cutoff)
subset_predictions = predictions.iloc[:cutoff_index]

subset_predictions.to_csv('your_filtered_predictions.csv', index=False)
pd.set_option('display.max_rows',1000)

In [ ]:
    results = subset_predictions
    predicted_ones = results[results['prediction_label'] == 1]
    TP = (predicted_ones['AnyPaymentReceived'] == 1).sum()
    FP = (predicted_ones['AnyPaymentReceived'] == 0).sum()
    precision = TP / (TP + FP) if (TP + FP) > 0 else 0
    print(f'Precision: {precision:.2f}| Total Call Predictions (1): {len(predicted_ones)}| Correct Predictions: {TP}| Incorrect: {len(predicted_ones)-TP}')

In [ ]:
results[results['Offense_Count']<10].head(100)

In [ ]:
results[results.Make_y =='SEAT']

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

sns.set(style="whitegrid")

plt.figure(figsize=(20, 10))  
sns.histplot(sept['Price Point']) 

plt.title('Distribution of Data')
plt.xlabel('Value')
plt.ylabel('Frequency')

plt.show()